In [1]:
    from IPython import get_ipython
    from nbconvert.exporters.templateexporter import TemplateExporter
    from IPython.core.inputtransformer import InputTransformer
    from jinja2 import Environment
    from collections import UserList

    class Template(UserList, InputTransformer):
        environment = Environment()
        def __init__(self, user_ns=None): 
            if user_ns is None:
                user_ns = get_ipython().user_ns
            self.user_ns = super().__init__() or user_ns
            get_ipython().input_transformer_manager.python_line_transforms.insert(0, self)
            
        def push(self, line): self.data.append(line)
        
        def reset(self, *, str=""""""):
            while self.data: str += self.data.pop(0) + "\n"
            return self.environment.from_string(str).render(**self.user_ns)

        def __del__(self): 
            get_ipython().python_line_transforms = [
                _ for _ in get_ipython().python_line_transforms
                if _ is not self]
        
        def __enter__(self, user_ns=None):
            if user_ns: self.user_ns = user_ns
            
        def __exit__(self, *_): del self

In [2]:
    def load_ipython_extension(ip):
        global TRANSFORMER
        TRANSFORMER = Template()
        
    def unload_ipython_extension(ip):
        global TRANSFORMER
        del TRANSFORMER